In [ ]:
"""
Created on Thu Oct 24 09:54 2024

Try to prepare grid for remapcon using xesmf

@author: Clara Burgard
"""

In [1]:
import xarray as xr
import xesmf
import numpy as np

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
inputpath = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [4]:
# Load your curvilinear grid file
ds = xr.open_dataset(inputpath + 'grid_eORCA1_T.nc')

In [5]:
# Assuming the dataset has 'lat' and 'lon' variables
lat = ds['gphit']  # 2D array of latitudes
lon = ds['glamt'] # 2D array of longitudes
lon360 = lon.where(lon>0,lon+360)

In [6]:
meshlon_ll,meshlat_ll = (lon + lon.shift(x=-1))/2, (lat + lat.shift(x=-1))/2
meshlon_lr,meshlat_lr = (lon + lon.shift(x=1))/2, (lat + lat.shift(x=-1))/2
meshlon_ur,meshlat_ur = (lon + lon.shift(x=1))/2, (lat + lat.shift(x=1))/2
meshlon_ul,meshlat_ul = (lon + lon.shift(x=-1))/2, (lat + lat.shift(x=1))/2

In [7]:
meshlon360_ll,meshlat360_ll = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=-1))/2
meshlon360_lr,meshlat360_lr = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=-1))/2
meshlon360_ur,meshlat360_ur = (lon360 + lon360.roll(x=1))/2, (lat + lat.roll(x=1))/2
meshlon360_ul,meshlat360_ul = (lon360 + lon360.roll(x=-1))/2, (lat + lat.roll(x=1))/2

In [8]:
lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nvertex','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nvertex','y','x'])
lon360_bnds = xr.DataArray(data=np.array([meshlon360_ll, meshlon360_lr, meshlon360_ur, meshlon360_ul]), dims=['nvertex','y','x']) 

In [25]:
# region where longitude hops from 180 to -180
prob_region = (lon <= -140) | (lon >= 140) & (np.abs(lon_bnds) < 25)
new_lon_bnds = lon_bnds.where(~prob_region, lon360_bnds)
new_lon_bnds = new_lon_bnds.where(new_lon_bnds <= 180, new_lon_bnds - 360)

In [26]:
lon_bnds.sel(x=94,y=330)

<xarray.DataArray (nvertex: 4)>
array([-24.6609723 ,  99.06571026,  99.06571026, -24.6609723 ])
Dimensions without coordinates: nvertex

In [27]:
new_lon_bnds.isel(nvertex=0).plot()

In [28]:
NEMO_new_grid = xr.merge([new_lon_bnds.rename('lon_bnds'),
                          lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat}).drop_vars(['glamt','gphit'])

In [29]:
NEMO_new_grid['lon'].attrs['standard_name'] = 'longitude'
NEMO_new_grid['lon'].attrs['units'] = 'degrees_east'
NEMO_new_grid['lon'].attrs['bounds'] = 'lon_bnds'

NEMO_new_grid['lat'].attrs['standard_name'] = 'latitude'
NEMO_new_grid['lat'].attrs['units'] = 'degrees_north'
NEMO_new_grid['lat'].attrs['bounds'] = 'lat_bnds'


In [31]:
NEMO_new_grid[['lon','lat','lon_bnds','lat_bnds']].set_coords(
    ("lat_bnds", "lon_bnds")).transpose('y','x','nvertex').to_netcdf(inputpath + 'NEMO_grid_withbnds.nc')

In [32]:
Casimir_grid = xr.open_dataset('/data/cburgard/CASIMIR_SIMU/raw/n42openc/n42openc_00910101_01001231_1Y_grid_T.nc')


In [39]:
Casimir_grid['nav_lat'] = Casimir_grid['nav_lat'].where(Casimir_grid['nav_lat'] != -1,NEMO_new_grid['lat'])
Casimir_grid['nav_lon'] = Casimir_grid['nav_lon'].where(Casimir_grid['nav_lon'] != -1,NEMO_new_grid['lon'])

In [41]:
Casimir_grid['bounds_nav_lon'] = Casimir_grid['bounds_nav_lon'].where(Casimir_grid['bounds_nav_lon'] != -1,NEMO_new_grid['lon_bnds'])
Casimir_grid['bounds_nav_lat'] = Casimir_grid['bounds_nav_lat'].where(Casimir_grid['bounds_nav_lat'] != -1,NEMO_new_grid['lat_bnds'])

In [43]:
Casimir_grid.to_netcdf(inputpath + 'Casimir_NEMO_grid_modif.nc')

In [45]:
Casimir_grid['nav_lon'].plot()

In [44]:
Casimir_grid['bounds_nav_lon'].isel(nvertex=0).plot()

In [ ]:
lon_bnds.where(lat_bnds == 0).isel(nvertex=2).plot()

In [ ]:
lon_bnds.where((lon >= -179.95) & (lon <= 179.95)).isel(nvertex=1).plot() #(lon_bnds > 4) &

In [ ]:
lon_bnds.isel(nvertex=1).plot()

In [ ]:
lon.max()